In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

In [12]:

import numpy as np
import struct
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import classification_report
from sklearn.svm import SVC

import pandas as pd



   
def fetch_mnist(mnist_dir,data_type):
    train_data_path = mnist_dir + 'train-images-idx3-ubyte.gz'
    train_label_path = mnist_dir + 'train-labels-idx1-ubyte.gz'
    test_data_path = mnist_dir + 't10k-images-idx3-ubyte.gz'
    test_label_path = mnist_dir + 't10k-labels-idx1-ubyte.gz'

    # train_img
    with open(train_data_path, 'rb') as f:
        data = f.read(16)
        des,img_nums,row,col = struct.unpack_from('>IIII', data, 0)
        train_x = np.zeros((img_nums, row*col))
        for index in range(img_nums):
            data = f.read(784)
            if len(data) == 784:
                train_x[index,:] = np.array(struct.unpack_from('>' + 'B' * (row * col), data, 0)).reshape(1,784)
        f.close()
    # train label
    with open(train_label_path, 'rb') as f:
        data = f.read(8)
        des,label_nums = struct.unpack_from('>II', data, 0)
        train_y = np.zeros((label_nums, 1))
        for index in range(label_nums):
            data = f.read(1)
            train_y[index,:] = np.array(struct.unpack_from('>B', data, 0)).reshape(1,1)
        f.close()

        # test_img
        with open(test_data_path, 'rb') as f:
            data = f.read(16)
            des, img_nums, row, col = struct.unpack_from('>IIII', data, 0)
            test_x = np.zeros((img_nums, row * col))
            for index in range(img_nums):
                data = f.read(784)
                if len(data) == 784:
                    test_x[index, :] = np.array(struct.unpack_from('>' + 'B' * (row * col), data, 0)).reshape(1, 784)
            f.close()
        # test label
        with open(test_label_path, 'rb') as f:
            data = f.read(8)
            des, label_nums = struct.unpack_from('>II', data, 0)
            test_y = np.zeros((label_nums, 1))
            for index in range(label_nums):
                data = f.read(1)
                test_y[index, :] = np.array(struct.unpack_from('>B', data, 0)).reshape(1, 1)
            f.close()
        if data_type == 'train':
            return train_x, train_y
        elif data_type == 'test':
            return test_x, test_y
        elif data_type == 'all':
            return train_x, train_y,test_x, test_y
        else:
            print('type error')

if __name__ == '__main__':
    train_data,train_lab,test_data,test_lab=fetch_mnist("data/data65/",'all')
    # 1.读取MNIST数据集并存储为numpy矩阵
    # train_img_data = read_file(train_img)
    # train_lab_data = read_file(train_lab)
    # test_img_data = read_file(test_img)
    # test_lab_data = read_file(test_lab)
    # train_data=process_img(train_img_data)
    # train_lab=process_lab(train_lab_data)
    # test_data=process_img(test_img_data[:1000])
    # test_lab=process_lab(test_lab_data[:1000])
    


    clf=SVC().fit(train_data,pd.DataFrame(train_lab).values.ravel())
    clf=SVC(kernel="linear").fit(train_data,pd.DataFrame(train_lab).values.ravel())
    predictions_labels=clf.predict(train_data)



    print(u'预测结果:')
    print(predictions_labels)
    print(u'算法评价:')
    print(classification_report(test_lab, predictions_labels))


    # train_data=[]
    # for i in train_img_data:
    #     image = cv2.imdecode(np.fromfile(i, dtype=np.uint8), cv2.IMREAD_COLOR)
    #     img = cv2.resize(image, (28, 28),
    #                  interpolation=cv2.INTER_CUBIC)

    #     hist=cv2.calcHist([img],[0,1],None,[28,28],[0.0,255.0,0.0,255.0])
    #     train_data.append(((hist/255).flatten()))
    
    # test_data=[]
    # for i in test_img_data:
    #     image=cv2.imdecode(np.fromfile(i,dtype=np.uint8),cv2.IMREAD_COLOR)


ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [1]:
import numpy as np
import struct
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import classification_report
from sklearn.svm import SVC
train_img = 'data/data65/train-images-idx3-ubyte.gz' #训练集图像
train_lab = 'data/data65/train-labels-idx1-ubyte.gz' #训练集标签
test_img = 'data/data65/t10k-images-idx3-ubyte.gz'   #测试集图像
test_lab = 'data/data65/t10k-labels-idx1-ubyte.gz'   #测试集标签

def read_file(idx_ubyte_file):
    with open(idx_ubyte_file, 'rb') as idxf:
        f = idxf.read()
    return f
    
def process_img(img_data):
    
    #解析文件头信息，魔数、图片数量、图片高、宽
    pointer = 0
    header = '>iiii'   #采用i格式读取前4行数据
    magic_num, images_num, height, width = struct.unpack_from(header, img_data, pointer)
    print("魔数：{} 图片数量：{} 图片高：{} 图片宽：{}".format(magic_num, images_num, height, width))
    
    #解析数据集
    img_size = height * width
    pro_way = '>' + str(img_size) + 'B'  #像素值格式为B，要读取784（图片大小）个像素值
    pointer += struct.calcsize(header)  #移动读取位置的指针
    imgs = np.empty((images_num, height, width))
    for i in range(images_num):
        if (i+1) % 10000 == 0:
            print("解析数量{}张".format(i+1))
        img_t = struct.unpack_from(pro_way, img_data, pointer)  #python自带的struct.unpack_from函数打开
        imgs[i] = np.array(img_t).reshape(height, width)
        pointer += struct.calcsize(pro_way)
    return imgs
        
def process_lab(lab_data):
    
    #解析文件头信息，魔数、标签数
    pointer = 0
    header = '>ii'
    magic_num, images_num = struct.unpack_from(header, lab_data, pointer)
    print("魔数：{} 图片数量：{}".format(magic_num, images_num))
    
    #解析数据集
    pro_way = '>B'
    pointer += struct.calcsize(header)
    labs = np.empty(images_num)
    for i in range(images_num):
        if (i+1) % 10000 == 0:
            print("解析数量{}张".format(i+1))
        img_t = struct.unpack_from(pro_way, lab_data, pointer)[0]
        labs[i] = img_t
        pointer += struct.calcsize(pro_way)
    return labs

if __name__ == '__main__':
    
    # 1.读取MNIST数据集并存储为numpy矩阵
    train_img_data = read_file(train_img)
    train_lab_data = read_file(train_lab)
    test_img_data = read_file(test_img)
    test_lab_data = read_file(test_lab)
    train_data=process_img(train_img_data)
    train_lab=process_lab(train_lab_data)
    test_data=process_img(test_img_data[:1000])
    test_lab=process_lab(test_lab_data[:1000])
    


    clf=SVC().fit(train_data,train_lab)
    clf=SVC(kernel="linear").fit(train_data,train_lab)
    predictions_labels=clf.predict(train_data)



    print(u'预测结果:')
    print(predictions_labels)
    print(u'算法评价:')
    print(classification_report(test_lab, predictions_labels))

魔数：529205256 图片数量：2055376946 图片高：226418 图片宽：1634299437


ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 